# Explorative analysis of charging stations in Germany
This notebook explores the current distribution of public charging infrastructure in Germany with some basic statistics and visualizations

## Imports and loading data 

In [4]:
import os,sys 
sys.path.insert(0,"../code")
import numpy as np 
import pandas as pd 
import geopandas as gpd 

In [24]:
# Download data 
! curl -o charging_stations_ger.xlsx "https://www.bundesnetzagentur.de/SharedDocs/Downloads/DE/Sachgebiete/Energie/Unternehmen_Institutionen/HandelundVertrieb/Ladesaeulen/Ladesaeulenkarte_Datenbankauszug22.xlsx;jsessionid=6BF61D44DDE921ABDF030645C9FBE18D?__blob=publicationFile&v=3"


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 87 1012k   87  884k    0     0  1566k      0 --:--:-- --:--:-- --:--:-- 1563k
100 1012k  100 1012k    0     0  1713k      0 --:--:-- --:--:-- --:--:-- 1710k


In [31]:
def move_data(path_save=r"data/"): 
    if not os.path.exists(path_save): 
        os.mkdir(path_save)
    if os.path.exists("charging_stations_ger.xlsx"): 
        os.rename("charging_stations_ger.xlsx", os.path.join(path_save, "charging_stations_ger.xlsx"))

move_data()

In [65]:
def excel_to_df(path_excel): 
    conv_dict = {"Anschlussleistung [kW]": lambda val: np.nan if "x" in val  
                 or "," in val or "/" in val else float(val)}
    rename_cols = { "Betreiber": "operator", 
                    "Adresse":  "address", 
                    "Postleitzahl Ort": "area_code", 
                    "Art der Ladeeinrichtung": "type_cs",
                    "Inbetriebnahmedatum": "date_launch", 
                    "Längengrad [DG]": "y_latlon", 
                    "Breitengrad [DG]": "x_latlon", 
                    "Anzahl Ladepunkte": "n_chargingpoints", 
                    "Anschlussleistung [kW]": "P_install_kW", 
                    **{f"Steckertypen{i}": f"type_charger_{i}" for i in range(1,5)},
                    **{f"P{i} [kW]": f"P_charger_{i}_kW" for i in range(1,5)}
                  }
    df_cs = (pd.read_excel("data/charging_stations_ger.xlsx", 
                           skiprows=5, 
                           parse_dates=[6], 
                           converters=conv_dict)
             .rename(columns= rename_cols)
            )[list(rename_cols.values())]
    return df_cs

df_cs = excel_to_df(path_excel="data/charging_stations_ger.xlsx") 
df_cs.head()

,operator,address,area_code,type_cs,date_launch,y_latlon,x_latlon,n_chargingpoints,P_install_kW,type_charger_1,type_charger_2,type_charger_3,type_charger_4,P_charger_1_kW,P_charger_2_kW,P_charger_3_kW,P_charger_4_kW
0,DREWAG Stadtwerke Dresden GmbH,Waltherstraße 9,01067 Dresden,Normalladeeinrichtung,2019-02-18,13.712142,51.060250,4,33.0,AC Steckdose Typ 2,AC Schuko,AC Steckdose Typ 2,AC Schuko,22.0,3.7,22.0,3.7
1,Autohaus Melkus GmbH,Hamburger Straße 30,01067 Dresden,Normalladeeinrichtung,2019-01-28,13.698890,51.059870,2,44.0,AC Steckdose Typ 2,AC Steckdose Typ 2,NaN,NaN,22.0,22,NaN,NaN
2,DREWAG Stadtwerke Dresden GmbH,Ringstraße 0,01067 Dresden,Schnellladeeinrichtung,2018-10-30,13.744003,51.048741,2,150.0,"DC Kupplung Combo, DC CHAdeMO",NaN,NaN,NaN,75.0,NaN,NaN,NaN
3,Volkswagen Sachsen GmbH; Die Gläserne Manufaktur,Lennéstraße 1,01069 Dresden,Schnellladeeinrichtung,2016-10-01,13.754270,51.043680,2,100.0,"DC Kupplung Combo, DC CHAdeMO",AC Kupplung Typ 2,NaN,NaN,50.0,43,NaN,NaN
4,Volkswagen Sachsen GmbH; Die Gläserne Manufaktur,Lennéstraße 1,01069 Dresden,Schnellladeeinrichtung,2016-10-01,13.754230,51.043650,2,100.0,"DC Kupplung Combo, DC CHAdeMO",AC Kupplung Typ 2,NaN,NaN,50.0,43,NaN,NaN


In [52]:
df_cs.describe()

,y_latlon,x_latlon,n_chargingpoints,P_charger_1_kW,P_charger_3_kW
count,8446.000000,8446.000000,8446.000000,8446.000000,431.000000
mean,9.772859,50.752239,1.981530,26.503068,21.527842
std,2.026088,1.849053,0.528537,33.382013,18.005470
min,5.897660,47.358000,1.000000,2.000000,0.000000
25%,8.245350,49.142699,2.000000,22.000000,20.000000
50%,9.639696,50.794441,2.000000,22.000000,22.000000
75%,11.213457,52.298942,2.000000,22.000000,22.000000
max,15.592092,54.956030,4.000000,350.000000,150.000000


In [70]:
from shapely.geometry import Point
def df_to_gdf(df_cs): 
    "Convert regular df to geopandas DataFrame"
    df_cs['geometry'] = list(zip(df_cs['x_latlon'],df_cs['y_latlon']))
    df_cs['geometry'] = df_cs['geometry'].apply(Point)
    return gpd.GeoDataFrame(df_cs, geometry='geometry', crs={'init': "4326"})

df_to_gdf(df_cs)

,operator,address,area_code,type_cs,date_launch,y_latlon,x_latlon,n_chargingpoints,P_install_kW,type_charger_1,type_charger_2,type_charger_3,type_charger_4,P_charger_1_kW,P_charger_2_kW,P_charger_3_kW,P_charger_4_kW,geometry
0,DREWAG Stadtwerke Dresden GmbH,Waltherstraße 9,01067 Dresden,Normalladeeinrichtung,2019-02-18,13.712142,51.060250,4,33.0,AC Steckdose Typ 2,AC Schuko,AC Steckdose Typ 2,AC Schuko,22.0,3.7,22.0,3.7,POINT (51.06025 13.712142)
1,Autohaus Melkus GmbH,Hamburger Straße 30,01067 Dresden,Normalladeeinrichtung,2019-01-28,13.698890,51.059870,2,44.0,AC Steckdose Typ 2,AC Steckdose Typ 2,NaN,NaN,22.0,22,NaN,NaN,POINT (51.05987 13.69889)
2,DREWAG Stadtwerke Dresden GmbH,Ringstraße 0,01067 Dresden,Schnellladeeinrichtung,2018-10-30,13.744003,51.048741,2,150.0,"DC Kupplung Combo, DC CHAdeMO",NaN,NaN,NaN,75.0,NaN,NaN,NaN,POINT (51.048741 13.744003)
3,Volkswagen Sachsen GmbH; Die Gläserne Manufaktur,Lennéstraße 1,01069 Dresden,Schnellladeeinrichtung,2016-10-01,13.754270,51.043680,2,100.0,"DC Kupplung Combo, DC CHAdeMO",AC Kupplung Typ 2,NaN,NaN,50.0,43,NaN,NaN,POINT (51.04368 13.75427)
4,Volkswagen Sachsen GmbH; Die Gläserne Manufaktur,Lennéstraße 1,01069 Dresden,Schnellladeeinrichtung,2016-10-01,13.754230,51.043650,2,100.0,"DC Kupplung Combo, DC CHAdeMO",AC Kupplung Typ 2,NaN,NaN,50.0,43,NaN,NaN,POINT (51.04365 13.75423)
5,DREWAG Stadtwerke Dresden GmbH,Heinrichstraße 16,01097 Dresden,Normalladeeinrichtung,2019-03-21,13.739990,51.060209,4,33.0,AC Steckdose Typ 2,AC Schuko,AC Steckdose Typ 2,AC Schuko,22.0,"3,7",22.0,"3,7",POINT (51.060209 13.73999)
6,DREWAG Stadtwerke Dresden GmbH,Archivstraße 0,01097 Dresden,Normalladeeinrichtung,2019-02-18,13.749640,51.057566,4,33.0,AC Steckdose Typ 2,AC Schuko,AC Steckdose Typ 2,AC Schuko,22.0,3.7,22.0,3.7,POINT (51.057566 13.74964)
7,DREWAG Stadtwerke Dresden GmbH,Wigardstraße 2,01097 Dresden,Normalladeeinrichtung,2018-10-22,13.496400,51.575660,4,33.0,AC Steckdose Typ 2,AC Schuko,AC Steckdose Typ 2,AC Schuko,22.0,3.7,22.0,3.7,POINT (51.57566 13.4964)
8,Simmel AG,Antonstraße 2A,01097 Dresden,Normalladeeinrichtung,2017-12-15,13.746462,51.064071,2,22.0,AC Steckdose Typ 2,AC Steckdose Typ 2,NaN,NaN,22.0,22,NaN,NaN,POINT (51.0640713 13.7464616)
9,DREWAG Stadtwerke Dresden GmbH,Antonstraße 24,01097 Dresden,Schnellladeeinrichtung,2017-09-04,13.739641,51.064823,2,93.0,"DC Kupplung Combo, DC CHAdeMO","AC Kupplung Typ 2, AC Schuko",NaN,NaN,50.0,43,NaN,NaN,POINT (51.064823 13.739641)


In [69]:
list(zip(df_cs[['y_latlon',"x_latlon"]]))

[('y_latlon',), ('x_latlon',)]